In [1]:
# CELL 1 — Imports + Paths
import polars as pl
from pathlib import Path
import random

DATA_DIR = Path("dataset/dataset_after_preprocessing")

TX_PATH   = DATA_DIR / "purchase_2024_2025_01_full.parquet"
USER_PATH = DATA_DIR / "user_2024.parquet"
ITEM_PATH = DATA_DIR / "item_2024.parquet"

SEED = 42
random.seed(SEED)

K_NEG = 3  # số negative / (customer_id, month)


In [2]:
# CELL 2 — Load + chuẩn hoá key dtype
lf_tx   = pl.scan_parquet(TX_PATH).with_columns([
    pl.col("customer_id").cast(pl.Utf8).alias("customer_id"),
    pl.col("item_id").cast(pl.Utf8).alias("item_id"),
])

lf_user = pl.scan_parquet(USER_PATH).with_columns([
    pl.col("customer_id").cast(pl.Utf8).alias("customer_id"),
])

lf_item = pl.scan_parquet(ITEM_PATH).with_columns([
    pl.col("item_id").cast(pl.Utf8).alias("item_id"),
])


In [3]:
# CELL 3 — Chuẩn hoá tx: parse date + amount + month
lf_tx2 = (
    lf_tx
    .with_columns([
        pl.col("date_key").cast(pl.Utf8)
          .str.strptime(pl.Date, "%Y%m%d", strict=False)
          .alias("tx_date"),
    ])
    .with_columns([
        (pl.col("tx_date").dt.year() * 100 + pl.col("tx_date").dt.month()).alias("month"),
        ((pl.col("price") * pl.col("quantity")) - pl.col("discount"))
            .clip(0, None)
            .alias("amount"),
    ])
)

END_DATE = lf_tx2.select(pl.col("tx_date").max()).collect().item()
print("END_DATE =", END_DATE)


END_DATE = 2025-01-30


In [4]:
# CELL 4 — Positive pairs (label=1) có month (EAGER)
pos_pairs = (
    lf_tx2
    .select(["customer_id", "item_id", "month"])
    .unique()
    .with_columns(pl.lit(1).cast(pl.Int32).alias("label"))
    .collect()
)

print(pos_pairs.columns)
print(pos_pairs["label"].value_counts())


['customer_id', 'item_id', 'month', 'label']
shape: (1, 2)
┌───────┬──────────┐
│ label ┆ count    │
│ ---   ┆ ---      │
│ i32   ┆ u32      │
╞═══════╪══════════╡
│ 1     ┆ 31001051 │
└───────┴──────────┘


In [5]:
# CELL 5 — Negative sampling theo (customer_id, month)
df_user_month = pos_pairs.select(["customer_id", "month"]).unique()

all_items = lf_item.select("item_id").collect()["item_id"].to_list()

neg_cand = (
    df_user_month
    .select([
        pl.col("customer_id").repeat_by(K_NEG).explode().alias("customer_id"),
        pl.col("month").repeat_by(K_NEG).explode().alias("month"),
    ])
    .with_columns(
        pl.Series("item_id", random.choices(all_items, k=df_user_month.height * K_NEG))
    )
)

neg_pairs = (
    neg_cand
    .join(pos_pairs.select(["customer_id", "item_id", "month"]), on=["customer_id", "item_id", "month"], how="anti")
    .with_columns(pl.lit(0).cast(pl.Int32).alias("label"))
)

print(neg_pairs.columns)
print(neg_pairs["label"].value_counts())


['customer_id', 'month', 'item_id', 'label']
shape: (1, 2)
┌───────┬──────────┐
│ label ┆ count    │
│ ---   ┆ ---      │
│ i32   ┆ u32      │
╞═══════╪══════════╡
│ 0     ┆ 20263705 │
└───────┴──────────┘


In [6]:
# CELL 6 — Gộp pos + neg -> df_pairs, lf_pairs (FIX schema mismatch)

import polars as pl

# 1) ép đúng thứ tự cột (quan trọng nhất để concat không lỗi)
pos_pairs2 = pos_pairs.select(["customer_id", "item_id", "month", "label"])
neg_pairs2 = neg_pairs.select(["customer_id", "item_id", "month", "label"])

# 2) ép dtype đồng nhất (phòng lỗi lặt vặt)
pos_pairs2 = pos_pairs2.with_columns([
    pl.col("customer_id").cast(pl.Utf8),
    pl.col("item_id").cast(pl.Utf8),
    pl.col("month").cast(pl.Int32),
    pl.col("label").cast(pl.Int32),
])

neg_pairs2 = neg_pairs2.with_columns([
    pl.col("customer_id").cast(pl.Utf8),
    pl.col("item_id").cast(pl.Utf8),
    pl.col("month").cast(pl.Int32),
    pl.col("label").cast(pl.Int32),
])

# 3) concat
df_pairs = pl.concat([pos_pairs2, neg_pairs2], how="vertical", rechunk=True)

# 4) lazy view để dùng tiếp pipeline
lf_pairs = df_pairs.lazy()

# 5) check nhanh
print("df_pairs shape:", df_pairs.shape)
print(df_pairs["label"].value_counts().sort("label"))
print("columns:", df_pairs.columns)


df_pairs shape: (51264756, 4)
shape: (2, 2)
┌───────┬──────────┐
│ label ┆ count    │
│ ---   ┆ ---      │
│ i32   ┆ u32      │
╞═══════╪══════════╡
│ 0     ┆ 20263705 │
│ 1     ┆ 31001051 │
└───────┴──────────┘
columns: ['customer_id', 'item_id', 'month', 'label']


In [16]:
# CELL 7 — Static features: item + user
item_static = lf_item.select(["item_id", "brand", "category_l1", "price", "age_group"])

# ưu tiên age_group từ user (nếu có)
user_static = lf_user.select([c for c in ["customer_id", "age_group"] if c in lf_user.schema])

lf_base = (
    lf_pairs
    .join(item_static, on="item_id", how="left")
    .join(user_static, on="customer_id", how="left")
)


C:\Users\DELL\AppData\Local\Temp\ipykernel_27392\3550168579.py:5: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  user_static = lf_user.select([c for c in ["customer_id", "age_group"] if c in lf_user.schema])


In [17]:
# CELL 8 — User history features
user_hist = (
    lf_tx2.group_by("customer_id")
    .agg([
        pl.len().alias("user_tx_cnt"),
        pl.col("amount").mean().alias("user_avg_spend"),
        pl.col("tx_date").max().alias("last_tx_date"),
    ])
    .with_columns(
        (pl.lit(END_DATE) - pl.col("last_tx_date")).dt.total_days().alias("recency_days")
    )
)


In [18]:
# CELL 9 — Item popularity + repurchase count
item_pop = lf_tx2.group_by("item_id").agg(pl.len().alias("item_popularity"))

repurchase = (
    lf_tx2.group_by(["customer_id", "item_id"])
    .agg(pl.len().alias("repurchase_cnt"))
)


In [19]:
# CELL 10 — Affinity features (brand/category/age_group(item))
need_cols = ["item_id", "brand", "category_l1"]
if "age_group" in lf_item.schema:
    need_cols.append("age_group")

lf_tx_item = lf_tx2.join(lf_item.select(need_cols), on="item_id", how="left")

brand_aff = (
    lf_tx_item.group_by(["customer_id", "brand"])
    .agg(pl.len().alias("brand_affinity"))
)

cat_aff = (
    lf_tx_item.group_by(["customer_id", "category_l1"])
    .agg(pl.len().alias("category_affinity"))
)

if "age_group" in lf_item.schema:
    age_aff = (
        lf_tx_item.group_by(["customer_id", "age_group"])
        .agg(pl.len().alias("agegroup_affinity"))
    )
else:
    age_aff = None


C:\Users\DELL\AppData\Local\Temp\ipykernel_27392\2551382302.py:3: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  if "age_group" in lf_item.schema:
C:\Users\DELL\AppData\Local\Temp\ipykernel_27392\2551382302.py:18: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  if "age_group" in lf_item.schema:


In [20]:
# CELL 11 — Assemble full feature table (pos+neg)
lf_feat = (
    lf_base
    .join(user_hist, on="customer_id", how="left")
    .join(item_pop, on="item_id", how="left")
    .join(repurchase, on=["customer_id", "item_id"], how="left")
    .join(brand_aff, on=["customer_id", "brand"], how="left")
    .join(cat_aff, on=["customer_id", "category_l1"], how="left")
)

if age_aff is not None:
    lf_tx_item_age = (
        lf_tx_item
        .select(["customer_id", "item_id", "age_group"])
        .unique()
        .rename({"age_group": "item_age_group"})
    )
    age_aff2 = age_aff.rename({"age_group": "item_age_group"})

    lf_feat = (
        lf_feat
        .join(lf_tx_item_age, on=["customer_id", "item_id"], how="left")
        .join(age_aff2, on=["customer_id", "item_age_group"], how="left")
        .drop("item_age_group")
    )
else:
    lf_feat = lf_feat.with_columns(pl.lit(0).alias("agegroup_affinity"))

# fill null cho numeric
lf_feat = lf_feat.with_columns([
    pl.col("user_tx_cnt").fill_null(0),
    pl.col("user_avg_spend").fill_null(0),
    pl.col("recency_days").fill_null(999999),
    pl.col("item_popularity").fill_null(0),
    pl.col("repurchase_cnt").fill_null(0),
    pl.col("brand_affinity").fill_null(0),
    pl.col("category_affinity").fill_null(0),
    pl.col("agegroup_affinity").fill_null(0),
])


In [21]:
# CELL 12 — Filter month CUỐI: giữ 202402 và 202409..202501
lf_feat = lf_feat.filter(
    (pl.col("month") == 202402) |
    ((pl.col("month") >= 202409) & (pl.col("month") <= 202501))
)


In [22]:
# CELL 13 — Select output columns (defensive)
cols_out = [
    "customer_id", "item_id", "label",
    "brand", "category_l1", "age_group", "price",
    "user_tx_cnt", "user_avg_spend", "last_tx_date", "recency_days",
    "item_popularity", "repurchase_cnt",
    "brand_affinity", "category_affinity", "agegroup_affinity",
    "month",
]

existing = set(lf_feat.collect_schema().names())   # an toàn hơn lf_feat.schema
cols_out = [c for c in cols_out if c in existing]

print("Missing cols:", [c for c in [
    "age_group"
] if c not in existing])

lf_feat = lf_feat.select(cols_out)


Missing cols: []


In [14]:
lf_pairs.select([
    pl.col("customer_id").dtype,
    pl.col("item_id").dtype,
]).limit(1).collect()


AttributeError: 'Expr' object has no attribute 'dtype'

In [23]:
# CELL 14 — Save (khuyên dùng: collect -> write_parquet để tránh Lazy plan lỗi)
OUT_PATH = DATA_DIR / "features_feb2024_sep2024_to_jan2025_withneg.parquet"

df_feat = lf_feat.collect(engine="streaming")
df_feat.write_parquet(OUT_PATH)

print("Saved:", OUT_PATH)
print("Shape:", df_feat.shape)
print(df_feat["label"].value_counts())


Saved: dataset\dataset_after_preprocessing\features_feb2024_sep2024_to_jan2025_withneg.parquet
Shape: (21697473, 17)
shape: (2, 2)
┌───────┬──────────┐
│ label ┆ count    │
│ ---   ┆ ---      │
│ i32   ┆ u32      │
╞═══════╪══════════╡
│ 0     ┆ 8572074  │
│ 1     ┆ 13125399 │
└───────┴──────────┘


In [24]:
import polars as pl

OUT_PATH = DATA_DIR / "features_feb2024_sep2024_to_jan2025_withneg.parquet"
df = pl.read_parquet(OUT_PATH)

print(df.shape)
print(df.columns)


(21697473, 17)
['customer_id', 'item_id', 'label', 'brand', 'category_l1', 'age_group', 'price', 'user_tx_cnt', 'user_avg_spend', 'last_tx_date', 'recency_days', 'item_popularity', 'repurchase_cnt', 'brand_affinity', 'category_affinity', 'agegroup_affinity', 'month']


In [25]:
df["label"].value_counts().sort("label")


label,count
i32,u32
0,8572074
1,13125399


In [26]:
df.select(pl.col("month").unique().sort())


month
i32
202402
202409
202410
202411
202412
202501


In [27]:
sample_user = df.select("customer_id").unique().sample(1).item()

df.filter(pl.col("customer_id") == sample_user).head(10)


customer_id,item_id,label,brand,category_l1,age_group,price,user_tx_cnt,user_avg_spend,last_tx_date,recency_days,item_popularity,repurchase_cnt,brand_affinity,category_affinity,agegroup_affinity,month
str,str,i32,str,str,str,"decimal[38,4]",u32,f64,date,i64,u32,u32,u32,u32,u32,i32
"""6774594""","""0076000000040""",1,"""Animo""","""Babycare""","""Từ 6M""",99000.0000,16,82877.5,2024-11-29,62,11710,1,2,3,2,202411
"""6774594""","""7115000000002""",1,"""Grinny""","""Thực phẩm cho bé""","""Từ 9M""",49000.0000,16,82877.5,2024-11-29,62,44747,2,3,11,3,202411
"""6774594""","""7115000000005""",1,"""Grinny""","""Thực phẩm cho bé""","""Từ 9M""",49000.0000,16,82877.5,2024-11-29,62,55520,1,3,11,3,202411
"""6774594""","""6268000000001""",1,"""Kutieskin""","""Hóa mỹ phẩm cho bé""","""Không xác định""",96000.0000,16,82877.5,2024-11-29,62,56759,1,1,1,6,202402
"""6774594""","""7092000000002""",1,"""Juqi""","""Thực phẩm cho bé""","""Từ 3Y""",40000.0000,16,82877.5,2024-11-29,62,5432,1,1,11,1,202411
"""6774594""","""0029120000031""",1,"""Happy Bites""","""Thực phẩm cho bé""","""Không xác định""",69000.0000,16,82877.5,2024-11-29,62,63929,1,1,11,6,202411
"""6774594""","""2278000000028""",0,"""Molfix""","""Tã""","""Không xác định""",294000.0000,16,82877.5,2024-11-29,62,51614,0,0,1,0,202402
"""6774594""","""5649000000001""",0,"""Không xác định""","""Thực phẩm cho gia đình""","""Không xác định""",249000.0000,16,82877.5,2024-11-29,62,0,0,0,0,0,202411
"""6774594""","""3406008200001""",0,"""Không xác định""","""Thời trang""","""Không xác định""",49000.0000,16,82877.5,2024-11-29,62,0,0,0,0,0,202411


In [28]:
df.filter(pl.col("label") == 0).head(5)


customer_id,item_id,label,brand,category_l1,age_group,price,user_tx_cnt,user_avg_spend,last_tx_date,recency_days,item_popularity,repurchase_cnt,brand_affinity,category_affinity,agegroup_affinity,month
str,str,i32,str,str,str,"decimal[38,4]",u32,f64,date,i64,u32,u32,u32,u32,u32,i32
"""2463329""","""4656000000001""",0,"""M'Smarty""","""TPCN""","""Không xác định""",195000.0000,19,69533.937321,2024-12-22,39,9659,1,1,1,14,202411
"""394640""","""2808000000001""",0,"""Yakult""","""Thực phẩm cho bé""","""Không xác định""",27000.0000,57,830365.418765,2024-12-20,41,127271,1,1,14,30,202410
"""6358928""","""3051000000002""",0,"""Thuyền Xưa""","""Thực phẩm cho bé""","""Không xác định""",55000.0000,134,170496.641794,2024-12-11,50,75261,1,2,52,90,202412
"""7568897""","""3518000000109""",0,"""Không xác định""","""Thời trang""","""Không xác định""",269000.0000,20,106619.31069,2024-09-19,133,187,1,1,2,13,202409
"""2562859""","""3788000000023""",0,"""MAM""","""Babycare""","""0-2M""",258000.0000,5,600.0,2024-12-31,30,1098,1,4,4,1,202412


In [30]:
null_df = df.select([
    pl.col(c).null_count().alias(c)
    for c in df.columns
]).transpose(include_header=True)

null_df


column,column_0
str,u32
"""customer_id""",0
"""item_id""",0
"""label""",0
"""brand""",1843
"""category_l1""",1843
…,…
"""repurchase_cnt""",0
"""brand_affinity""",0
"""category_affinity""",0
